In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.applications import DenseNet121
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,MaxPooling2D , Conv2D,Flatten
from tensorflow.keras.models import Sequential

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report


from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import glob
import zipfile

In [ ]:
#file = r'C:\Users\pavit\OneDrive\Desktop\data\Train.zip'

In [ ]:
#with zipfile.ZipFile(file, 'r') as zip_ref:
#    zip_ref.extractall('data/raw')

In [ ]:
#file = r'C:\Users\pavit\OneDrive\Desktop\data\Test.zip'

In [ ]:
#with zipfile.ZipFile(file, 'r') as zip_ref:
#    zip_ref.extractall('data/raw')

In [ ]:
# import requests
# import zipfile

# # Replace 'test_shared_link_here' with the actual shareable link for "Test.zip"
# test_url = 'https://drive.google.com/drive/u/0/recent'

# test_response = requests.get(test_url)

# # Replace 'Test.zip' with the desired file name
# with open('Test.zip', 'wb') as file:
#     file.write(test_response.content)

# # Extract "Test.zip" to a specific folder
# with zipfile.ZipFile('Test.zip', 'r') as test_zip_ref:
#     test_zip_ref.extractall('path_to_test_extraction_folder')


In [ ]:
train_dir = r"C:\Users\pavit\Downloads\data\Train"
test_dir = r"C:\Users\pavit\Downloads\data\Test"

SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 64
EPOCHS = 1
LR =  0.00003

In [ ]:
crime_types=os.listdir(train_dir)
n=len(crime_types)
print("Number of crime categories : ",n)

In [ ]:
crimes={}
train=test=0
for clss in crime_types:
    num=len(os.listdir(os.path.join(train_dir,clss)))
    train+=num
    test+=len(os.listdir(os.path.join(test_dir,clss)))
    
    crimes[clss]=num

In [ ]:
plt.figure(figsize=(10, 7))
plt.pie(x=np.array([train,test]), autopct="%.1f%%", explode=[0.1, 0.1], labels=["Training Data", "Test Data"], pctdistance=0.5)
plt.title("Share of train and test images ", fontsize=14);

In [ ]:
plt.figure(figsize=(7, 6))
plt.barh(list(crimes.keys()), list(crimes.values()), height=0.6, align="center")
plt.yticks(rotation=0)

plt.xlabel("Frame count", fontsize=14)
plt.ylabel("Classification", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(7,10))
plt.pie(x=np.array(list(crimes.values())), autopct="%.1f%%", explode=[0.1]*n,labels=list(crimes.keys()), pctdistance=0.5)
plt.title("Share of train and test images ", fontsize=14);

In [ ]:
IMG_WIDTH=64
IMG_HEIGHT=64

IMG_SHAPE=(IMG_HEIGHT,IMG_WIDTH)
seed=69

INPUT_SHAPE=(IMG_HEIGHT,IMG_WIDTH,3)

In [ ]:
train_set=image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SHAPE,
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset="training",
)

val_set=image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SHAPE,
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset="validation",
)

test_set=image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    class_names=None,
    batch_size=BATCH_SIZE,
    image_size=IMG_SHAPE,
    shuffle=False,
    seed=seed,
)


In [ ]:
def transfer_learning():
    base_model=DenseNet121(include_top=False,input_shape=INPUT_SHAPE,weights="imagenet")
    
    thr=149
    for layers in base_model.layers[:thr]:
        layers.trainable=False
    
    for layers in base_model.layers[thr:]:
        layers.trainable=True
    
    return base_model

In [ ]:
def create_model():
    model=Sequential()
    
    base_model=transfer_learning()
    model.add(base_model)
    
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.2))
    
    model.add(Dense(n,activation="softmax",name="classification"))
    
    model.summary()
    
    return model
    

In [ ]:
model=create_model()

model.compile(optimizer="adam", 
                loss='categorical_crossentropy',
                metrics = [tf.keras.metrics.AUC()])

In [ ]:
# history = model.fit(x = train_set,validation_data=val_set,epochs = EPOCHS)


In [ ]:
# Create a smaller subset of your data for testing
# Assuming you have a large dataset with X_train and y_train
# Create a smaller subset with 10% of the data
#subset_size = int(0.1 * len(train_set))
#train_set = train_set.take(subset_size)

# Train your model on the smaller subset
#history = model.fit(train_set, validation_data=val_set, epochs=EPOCHS)

In [ ]:
# Create a smaller subset of your data for testing
# Assuming you have a large dataset with X_train and y_train
# Create a smaller subset with 5% of the data
subset_size = int(0.05 * len(train_set))
train_set_subset = train_set.take(subset_size)

# Train your model on the smaller subset
history_subset = model.fit(train_set_subset, validation_data=val_set, epochs=EPOCHS)


In [ ]:
y_true =  np.array([])

for x, y in test_set:
  y_true = np.concatenate([y_true, np.argmax(y.numpy(), axis=-1)])


In [ ]:
y_pred=model.predict(test_set)


In [ ]:
y_pred


In [ ]:
y_true


In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    for (idx, c_label) in enumerate(crime_types):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr,lw=2, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'black',linestyle='dashed', lw=4, label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
fig, c_ax = plt.subplots(1,1, figsize = (5,8))


print('ROC AUC score:', multiclass_roc_auc_score(y_true , y_pred  , average = "micro"))
plt.xlabel('FALSE POSITIVE RATE', fontsize=18)
plt.ylabel('TRUE POSITIVE RATE', fontsize=16)
plt.legend(fontsize = 11.5)
plt.show()

In [ ]:
#CNN

In [ ]:
def create_custom_cnn_model(base_model):
    model = Sequential()
    
    # Add the transfer learning base model
    model.add(base_model)
    
    # Add additional convolutional layers
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    # Global average pooling and dense layers
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.2))
    
    # Output layer
    model.add(Dense(n, activation="softmax", name="classification"))
    
    model.summary()
    
    return model

In [ ]:
# base_model = transfer_learning()

# # Create a custom CNN model on top of the transfer learning base model
# custom_model = create_custom_cnn_model(base_model)



In [ ]:
def create_custom_cnn_model(base_model):
    model = Sequential()
    
    # Add the base model
    model.add(base_model)
    
    # Global Average Pooling Layer
    model.add(GlobalAveragePooling2D())
    
    # Dense layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    
    # Output layer
    model.add(Dense(n, activation='softmax', name='classification'))
    
    model.summary()
    
    return model

# Define and compile the base model (transfer learning)
base_model = transfer_learning()

# Create a custom CNN model on top of the transfer learning base model
custom_model = create_custom_cnn_model(base_model)

# Compile the custom model
custom_model.compile(optimizer="adam", 
                     loss='categorical_crossentropy',
                     metrics=[tf.keras.metrics.AUC()])


In [ ]:
IMG_HEIGHT = 32
IMG_WIDTH = 32
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

# Compile the custom model
custom_model.compile(optimizer="adam", 
                     loss='categorical_crossentropy',
                     metrics=[tf.keras.metrics.AUC()])

# Fine-tune the entire model on your dataset
fine_tune_epochs = 1  # Adjust the number of fine-tuning epochs as needed
custom_model.fit(train_set, validation_data=val_set, epochs=fine_tune_epochs)

In [ ]:
pip install Pillow


In [ ]:
import os
from PIL import Image

# Define the directory containing your image files
dataset_dir = r"C:\Users\pavit\Downloads\data\Train\Arrest"

# Get a list of image file names in the directory
image_files = [f for f in os.listdir(dataset_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

num_images_to_display = 5  # You can change this number to display a different number of images

for i in range(num_images_to_display):
    if i >= len(image_files):
        break
    image_file = os.path.join(dataset_dir, image_files[i])
    image = Image.open(image_file)
    image.show()


In [ ]:
print("Dataset Directory:", dataset_dir)
print("Image Files:", image_files)


# Fine tuning

In [ ]:
# Create a custom CNN model on top of the transfer learning base model
def create_custom_cnn_model(base_model):
    model = Sequential()

    # Add the base model
    model.add(base_model)

    # Global Average Pooling Layer
    model.add(GlobalAveragePooling2D())

    # Dense layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))

    # Output layer
    model.add(Dense(n, activation='softmax', name='classification'))

    model.summary()

    return model

# Define and compile the base model (transfer learning)
base_model = transfer_learning()

# Update the IMG_HEIGHT and IMG_WIDTH to 64
IMG_HEIGHT = 64
IMG_WIDTH = 64
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

# Create a new custom CNN model on top of the transfer learning base model
custom_model = create_custom_cnn_model(base_model)

# Compile the custom model
custom_model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC()])

# Fine-tune the entire model on your dataset
fine_tune_epochs = 3  # Adjust the number of fine-tuning epochs as needed
custom_model.fit(train_set, validation_data=val_set, epochs=fine_tune_epochs)


In [ ]:
test_metrics = custom_model.evaluate(test_set)
print("Test Loss:", test_metrics[0])
print("Test AUC:", test_metrics[1])


In [ ]:
y_true = np.concatenate([y_true, np.argmax(y.numpy(), axis=-1)])


In [ ]:
y_true = np.concatenate([y_true, np.argmax(y.numpy(), axis=-1)], axis=0)
